In [17]:
import numpy as np
import matplotlib.pyplot as plt

In [18]:
# Set a random seed for reproducibility
np.random.seed(42)

# Define the number of samples
k_max = 50000
t = np.linspace(0, k_max, k_max)  # Time variable

# Initialize u(k) and y(k) arrays with dtype float32
u = np.zeros(k_max, dtype=np.float32)
y = np.zeros(k_max, dtype=np.float32)

# Generate random inputs (float32)
u = (20 * np.random.random(k_max) - 10).astype(np.float32)

# Define y(k) calculations with dtype float32
for k in range(1, k_max):
    y[k] = np.float32(1 / (1 + (y[k-1])**2)) + np.float32(0.25 * u[k]) - np.float32(0.3 * u[k-1])

# Shift arrays for y(k-1), y(k-2), u(k-1), u(k-2)
y_k_1 = np.zeros(k_max, dtype=np.float32)
y_k_2 = np.zeros(k_max, dtype=np.float32)
u_k_1 = np.zeros(k_max, dtype=np.float32)
u_k_2 = np.zeros(k_max, dtype=np.float32)

y_k_1[1:] = y[:-1]
y_k_2[2:] = y[:-2]
u_k_1[1:] = u[:-1]
u_k_2[2:] = u[:-2]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(t, u, label='u(k)')
plt.xlabel('Time')
plt.ylabel('u(k)')
plt.title('Plot of u(k) over Time')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(t, y, label='y(k)')
plt.xlabel('Time')
plt.ylabel('y(k)')
plt.title('Plot of y(k) over Time')
plt.legend()
plt.grid(True)
plt.show()

In [21]:
def normalize(data):
    data = data.astype(np.float32)
    return 2 * (data - np.min(data, axis=0)) / (np.max(data, axis=0) - np.min(data, axis=0)) - 1

def denormalize(normalized_data, data_min, data_max):
    normalized_data = normalized_data.astype(np.float32)
    return (normalized_data + 1) * (data_max - data_min) / 2 + data_min

In [22]:
# Split the data into training and validation sets
split_ratio = 0.5
split_index = int(k_max * split_ratio)

X = np.array([y_k_1, y_k_2, u, u_k_1, u_k_2]).T
y = y.reshape(-1, 1)

X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y[:split_index], y[split_index:]

# Normalize inputs and outputs separately
u_norm = normalize(u)            # Normalize input u(k)
u_k_1_norm = normalize(u_k_1)    # Normalize u(k-1)
u_k_2_norm = normalize(u_k_2)    # Normalize u(k-2)

# Normalize and reshape y(k), y(k-1), and y(k-2)
y_norm = normalize(y).reshape(-1, 1)            # Normalize output y(k) and reshape to (k_max, 1)
y_k_1_norm = normalize(y_k_1).reshape(-1, 1)    # Normalize y(k-1) and reshape to (k_max, 1)
y_k_2_norm = normalize(y_k_2).reshape(-1, 1)    # Normalize y(k-2) and reshape to (k_max, 1)

# Combine normalized inputs into X (features matrix), no need to reshape X
X_norm = np.array([y_k_1_norm.flatten(), y_k_2_norm.flatten(), u_norm, u_k_1_norm, u_k_2_norm], dtype=np.float32).T

# Split the normalized data into training and validation sets
split_index = int(k_max * split_ratio)

X_train_norm = X_norm[:split_index]
X_val_norm = X_norm[split_index:]

y_train_norm = y_norm[:split_index]
y_val_norm = y_norm[split_index:]

In [23]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_sizes, output_size, learning_rate=0.01, l2_lambda=0.001):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.l2_lambda = l2_lambda  # L2 regularization parameter

        # Initialize weights and biases using Nguyen-Widrow initialization
        self.W = []
        self.b = []

        # Layer 1 (input to first hidden layer)
        self.W.append(self.nguyen_widrow_init(input_size, hidden_sizes[0]))
        self.b.append(np.zeros((1, hidden_sizes[0]), dtype=np.float32))

        # Hidden layers
        for i in range(1, len(hidden_sizes)):
            self.W.append(self.nguyen_widrow_init(hidden_sizes[i - 1], hidden_sizes[i]))
            self.b.append(np.zeros((1, hidden_sizes[i]), dtype=np.float32))

        # Output layer (last hidden layer to output)
        self.W.append(self.nguyen_widrow_init(hidden_sizes[-1], output_size))
        self.b.append(np.zeros((1, output_size), dtype=np.float32))

    def nguyen_widrow_init(self, input_size, output_size):
        """Nguyen-Widrow initialization."""
        W = np.random.randn(input_size, output_size).astype(np.float32) * 0.5
        beta = 0.7 * (output_size ** (1.0 / input_size))
        norm = np.linalg.norm(W, axis=0)
        return beta * W / norm

    def tanh(self, x):
        return np.tanh(x)

    def tanh_derivative(self, x):
        return 1 - np.tanh(x) ** 2

    def mse_loss(self, y_true, y_pred):
        return np.mean((y_true - y_pred) ** 2)

    def r2_score(self, y_true, y_pred):
        ss_res = np.sum((y_true - y_pred) ** 2)
        ss_tot = np.sum((y_true - np.mean(y_true)) ** 2) + 1e-4  # Prevent division by zero
        return 1 - (ss_res / ss_tot)

    def forward(self, X):
        # Forward pass through all layers
        self.activations = []  # Store activations for backpropagation
        self.Z = []  # Store pre-activation values

        # Input to first hidden layer
        Z = np.dot(X, self.W[0]) + self.b[0]
        A = self.tanh(Z)
        self.Z.append(Z)
        self.activations.append(A)

        # Hidden layers
        for i in range(1, len(self.hidden_sizes)):
            Z = np.dot(self.activations[-1], self.W[i]) + self.b[i]
            A = self.tanh(Z)
            self.Z.append(Z)
            self.activations.append(A)

        # Apply tanh at the output layer (for control system constraint)
        Z = np.dot(self.activations[-1], self.W[-1]) + self.b[-1]
        A = self.tanh(Z)
        self.Z.append(Z)
        self.activations.append(A)

        return A  # Final output after tanh activation

    def backward(self, X, y_true, y_pred):
        m = X.shape[0]  # Number of training samples

        # Gradient for output layer
        dZ = (y_pred - y_true) * self.tanh_derivative(self.Z[-1])  # Gradient of output layer with tanh derivative
        dW = np.dot(self.activations[-2].T, dZ) / m + (self.l2_lambda * self.W[-1] / m)  # L2 regularization
        db = np.sum(dZ, axis=0, keepdims=True) / m

        # Update weights and biases for output layer
        self.W[-1] -= self.learning_rate * dW
        self.b[-1] -= self.learning_rate * db

        # Backpropagate through the hidden layers
        for i in range(len(self.hidden_sizes) - 1, 0, -1):
            dA = np.dot(dZ, self.W[i + 1].T)
            dZ = dA * self.tanh_derivative(self.Z[i])
            dW = np.dot(self.activations[i - 1].T, dZ) / m + (self.l2_lambda * self.W[i] / m)
            db = np.sum(dZ, axis=0, keepdims=True) / m

            self.W[i] -= self.learning_rate * dW
            self.b[i] -= self.learning_rate * db

        # Gradient for the first hidden layer
        dA = np.dot(dZ, self.W[1].T)
        dZ = dA * self.tanh_derivative(self.Z[0])
        dW = np.dot(X.T, dZ) / m + (self.l2_lambda * self.W[0] / m)
        db = np.sum(dZ, axis=0, keepdims=True) / m

        # Update weights and biases for the first hidden layer
        self.W[0] -= self.learning_rate * dW
        self.b[0] -= self.learning_rate * db

    def train(self, X_train, y_train, X_val, y_val, epochs):
        # Create attributes for the losses and accuracies to be accessed later
        self.train_losses, self.val_losses = [], []
        self.train_r2_scores, self.val_r2_scores = [], []

        # Store predictions as attributes
        self.y_train_pred = None  # To store training predictions
        self.y_val_pred = None  # To store validation predictions

        print(f"{'Epoch':<10}{'Train Loss':<15}{'Train R²':<15}{'Val Loss':<15}{'Val R²':<15}")
        print("=" * 65)

        for epoch in range(epochs):
            # Forward pass (training set)
            self.y_train_pred = self.forward(X_train)  # Store training predictions

            # Compute training loss and R² score
            train_loss = self.mse_loss(y_train, self.y_train_pred)
            train_r2 = self.r2_score(y_train, self.y_train_pred)

            # Forward pass (validation set)
            self.y_val_pred = self.forward(X_val)  # Store validation predictions

            # Compute validation loss and R² score
            val_loss = self.mse_loss(y_val, self.y_val_pred)
            val_r2 = self.r2_score(y_val, self.y_val_pred)

            # Store metrics
            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)
            self.train_r2_scores.append(train_r2)
            self.val_r2_scores.append(val_r2)

            # Backward pass (training set)
            self.backward(X_train, y_train, self.y_train_pred)

            # Print progress every 100 epochs
            if (epoch + 1) % 100 == 0:
                print(f"{epoch + 1:<10}{train_loss:<15.4f}{train_r2:<15.4f}{val_loss:<15.4f}{val_r2:<15.4f}")

        print("=" * 65)
        print(f"Final Training Loss: {train_loss:.4f}, Final Training R²: {train_r2:.4f}")
        print(f"Final Validation Loss: {val_loss:.4f}, Final Validation R²: {val_r2:.4f}")

        # After training, plot the metrics
        self.plot_training_history(self.train_losses, self.val_losses, self.train_r2_scores, self.val_r2_scores)

    def plot_training_history(self, train_losses, val_losses, train_r2_scores, val_r2_scores):
        epochs_range = range(1, len(train_losses) + 1)

        plt.figure(figsize=(14, 6))

        # Loss plot
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, train_losses, label="Training Loss")
        plt.plot(epochs_range, val_losses, label="Validation Loss")
        plt.xlabel("Epochs")
        plt.ylabel("Loss (MSE)")
        plt.title("Training and Validation Loss")
        plt.legend()

        # R² Scores plot (Accuracy)
        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, train_r2_scores, label="Training R² Score")
        plt.plot(epochs_range, val_r2_scores, label="Validation R² Score")
        plt.xlabel("Epochs")
        plt.ylabel("R² Score")
        plt.title("Training and Validation R² Score")
        plt.legend()

        plt.show()


In [24]:
# Initialize the neural network
input_size = 5
hidden_sizes = [7, 7, 7, 7, 7]  # Five hidden layers with 7 neurons each
output_size = 1
learning_rate = 0.1  # Reduced learning rate for stability
l2_lambda = 0.1  # Regularization parameter
epochs = 5000

nn = NeuralNetwork(input_size, hidden_sizes, output_size, learning_rate, l2_lambda)

In [ ]:
# Train the network and print the losses and R² scores
nn.train(X_train_norm, y_train_norm, X_val_norm, y_val_norm, epochs)

In [ ]:
# Access the stored losses and accuracies
train_losses = nn.train_losses  # List of training losses
val_losses = nn.val_losses      # List of validation losses
train_r2_scores = nn.train_r2_scores  # List of training R² scores
val_r2_scores = nn.val_r2_scores      # List of validation R² scores

# Print the final losses and R² scores
print("Final Training Losses:", train_losses[-1])
print("Final Validation Losses:", val_losses[-1])
print("Final Training R² Scores:", train_r2_scores[-1])
print("Final Validation R² Scores:", val_r2_scores[-1])

In [ ]:
nn.plot_training_history(train_losses, val_losses, train_r2_scores, val_r2_scores)

In [ ]:
def plot_actual_vs_predicted(y_actual, y_pred, title="Actual vs Predicted"):
    plt.figure(figsize=(10, 6))
    
    # Plot actual values
    plt.plot(y_actual, label="Actual", color="blue")
    
    # Plot predicted values
    plt.plot(y_pred, label="Predicted", color="orange", linestyle='dashed')
    
    # Labels and title
    plt.xlabel("Samples")
    plt.ylabel("Output (y)")
    plt.title(title)
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Access the predictions for the training and validation sets
y_train_pred_norm = nn.y_train_pred  # Predicted values for the training set
y_val_pred_norm = nn.y_val_pred      # Predicted values for the validation set

# Denormalize the predictions
y_train_pred = denormalize(y_train_pred_norm, np.min(y_train), np.max(y_train))
y_val_pred = denormalize(y_val_pred_norm, np.min(y_val), np.max(y_val))

# Call the function to plot for training and validation
plot_actual_vs_predicted(y_train, y_train_pred, title="Training: Actual vs Predicted")
plot_actual_vs_predicted(y_val, y_val_pred, title="Validation: Actual vs Predicted")

In [ ]:
# Access weights and biases after training
weights = nn.W  # List of weight matrices for each layer
biases = nn.b   # List of bias vectors for each layer

# Print weights and biases for each layer
for i, (W, b) in enumerate(zip(weights, biases)):
    print(f"Layer {i + 1} Weights (W{i+1}):")
    print(W)
    print(f"Layer {i + 1} Biases (b{i+1}):")
    print(b)
    print("=" * 50)

In [30]:
class NeuralNetworkStochastic:
    def __init__(self, W, b, learning_rate=0.0001, l2_lambda=0.001, clip_value=5):
        self.W = W  # Predefined weights
        self.b = b  # Predefined biases
        self.learning_rate = learning_rate
        self.l2_lambda = l2_lambda  # L2 regularization parameter
        self.clip_value = clip_value  # For gradient clipping

        # Extract hidden sizes from the provided weight dimensions
        self.hidden_sizes = [W[i].shape[1] for i in range(len(W) - 1)]
        self.output_size = W[-1].shape[1]

    def tanh(self, x):
        return np.tanh(x)

    def tanh_derivative(self, x):
        return 1 - np.tanh(x) ** 2

    def mse_loss(self, y_true, y_pred):
        return np.mean((y_true - y_pred) ** 2)

    def r2_score(self, y_true, y_pred):
        ss_res = np.sum((y_true - y_pred) ** 2)
        ss_tot = np.sum((y_true - np.mean(y_true)) ** 2) + 1e-4  # Avoid division by zero
        return 1 - (ss_res / ss_tot)

    def forward(self, X):
        # Forward pass through all layers
        self.activations = []  # Store activations for backpropagation
        self.Z = []  # Store pre-activation values

        # Input to first hidden layer
        Z = np.dot(X, self.W[0]) + self.b[0]
        A = self.tanh(Z)
        self.Z.append(Z)
        self.activations.append(A)

        # Hidden layers
        for i in range(1, len(self.hidden_sizes)):
            Z = np.dot(self.activations[-1], self.W[i]) + self.b[i]
            A = self.tanh(Z)
            self.Z.append(Z)
            self.activations.append(A)

        # Output layer (tanh activation for control system constraint)
        Z = np.dot(self.activations[-1], self.W[-1]) + self.b[-1]
        self.Z.append(Z)
        A = self.tanh(Z)  # Apply tanh activation in the output layer
        self.activations.append(A)

        return A  # Final output with tanh activation

    def backward(self, X, y_true, y_pred):
        # Single data point (stochastic gradient descent)
        m = 1  # Only one data point for SGD

        # Gradient for output layer
        dZ = (y_pred - y_true) * self.tanh_derivative(self.Z[-1])  # Derivative of tanh at the output
        dW = np.dot(self.activations[-2].T, dZ) / m + (self.l2_lambda * self.W[-1] / m)
        db = np.sum(dZ, axis=0, keepdims=True) / m

        # Gradient clipping
        dW = np.clip(dW, -self.clip_value, self.clip_value)
        db = np.clip(db, -self.clip_value, self.clip_value)

        # Update weights and biases for output layer
        self.W[-1] -= self.learning_rate * dW
        self.b[-1] -= self.learning_rate * db

        # Backpropagate through the hidden layers
        for i in range(len(self.hidden_sizes) - 1, 0, -1):
            dA = np.dot(dZ, self.W[i + 1].T)
            dZ = dA * self.tanh_derivative(self.Z[i])
            dW = np.dot(self.activations[i - 1].T, dZ) / m + (self.l2_lambda * self.W[i] / m)
            db = np.sum(dZ, axis=0, keepdims=True) / m

            # Gradient clipping
            dW = np.clip(dW, -self.clip_value, self.clip_value)
            db = np.clip(db, -self.clip_value, self.clip_value)

            self.W[i] -= self.learning_rate * dW
            self.b[i] -= self.learning_rate * db

        # Gradient for the first hidden layer
        dA = np.dot(dZ, self.W[1].T)
        dZ = dA * self.tanh_derivative(self.Z[0])
        dW = np.dot(X.T, dZ) / m + (self.l2_lambda * self.W[0] / m)
        db = np.sum(dZ, axis=0, keepdims=True) / m

        # Gradient clipping
        dW = np.clip(dW, -self.clip_value, self.clip_value)
        db = np.clip(db, -self.clip_value, self.clip_value)

        # Update weights and biases for the first hidden layer
        self.W[0] -= self.learning_rate * dW
        self.b[0] -= self.learning_rate * db

    def train(self, X_train, y_train, X_val, y_val, epochs):
        # Initialize previous predictions for the first two iterations
        y_pred_k_1 = y_train[-1, 0]
        y_pred_k_2 = y_train[-2, 0]

        # Create attributes for the losses and accuracies to be accessed later
        self.train_losses, self.val_losses = [], []
        self.train_r2_scores, self.val_r2_scores = [], []

        # Store predictions as attributes
        self.y_train_pred = None  # To store training predictions
        self.y_val_pred = None  # To store validation predictions

        print(f"{'Epoch':<10}{'Train Loss':<15}{'Train R²':<15}{'Val Loss':<15}{'Val R²':<15}")
        print("=" * 65)

        for epoch in range(epochs):
            epoch_loss = 0

            # Training process (stochastic)
            for i in range(X_train.shape[0]):
                # For the first two data points, use previous known y values
                if i < 2:
                    X_input = np.array([X_train[i, 0], X_train[i, 1], X_train[i, 2], y_pred_k_1, y_pred_k_2]).reshape(1, -1)
                else:
                    # Use the predictions from previous steps
                    X_input = np.array([X_train[i, 0], X_train[i, 1], X_train[i, 2], y_pred_k_1, y_pred_k_2]).reshape(1, -1)

                # Forward pass
                y_pred = self.forward(X_input)

                # Compute loss
                loss = self.mse_loss(y_train[i:i+1], y_pred)
                epoch_loss += loss

                # Backward pass
                self.backward(X_input, y_train[i:i+1], y_pred)

                # Update previous predictions
                y_pred_k_2 = y_pred_k_1
                y_pred_k_1 = y_pred[0, 0]

            # Calculate average loss for training
            train_loss = epoch_loss / X_train.shape[0]

            # Validation pass
            self.y_val_pred = self.forward(X_val)  # Store validation predictions
            val_loss = self.mse_loss(y_val, self.y_val_pred)
            val_r2 = self.r2_score(y_val, self.y_val_pred)

            # Calculate R² for training data
            self.y_train_pred = self.forward(X_train)  # Store training predictions
            train_r2 = self.r2_score(y_train, self.y_train_pred)

            # Store metrics
            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)
            self.train_r2_scores.append(train_r2)
            self.val_r2_scores.append(val_r2)

            # Print metrics for the current epoch
            print(f"{epoch + 1:<10}{train_loss:<15.4f}{train_r2:<15.4f}{val_loss:<15.4f}{val_r2:<15.4f}")

            # Early stopping if loss is small enough
            if train_loss <= 1e-4:
                break

        print("=" * 65)
        print(f"Final Training Loss: {train_loss:.4f}, Final Training R²: {train_r2:.4f}")
        print(f"Final Validation Loss: {val_loss:.4f}, Final Validation R²: {val_r2:.4f}")

        # After training, plot the metrics
        self.plot_training_history(self.train_losses, self.val_losses, self.train_r2_scores, self.val_r2_scores)

    def plot_training_history(self, train_losses, val_losses, train_r2_scores, val_r2_scores):
        epochs_range = range(1, len(train_losses) + 1)

        plt.figure(figsize=(14, 6))

        # Loss plot
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, train_losses, label="Training Loss")
        plt.plot(epochs_range, val_losses, label="Validation Loss")
        plt.xlabel("Epochs")
        plt.ylabel("Loss (MSE)")
        plt.title("Training and Validation Loss")
        plt.legend()

        # R² Scores plot (Accuracy)
        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, train_r2_scores, label="Training R² Score")
        plt.plot(epochs_range, val_r2_scores, label="Validation R² Score")
        plt.xlabel("Epochs")
        plt.ylabel("R² Score")
        plt.title("Training and Validation R² Score")
        plt.legend()

        plt.show()

In [33]:
# Initialize the neural network
input_size = 5
hidden_sizes = [7, 7, 7, 7, 7]  # Five hidden layers with 7 neurons each
output_size = 1
learning_rate = 0.01  # Reduced learning rate for stability
l2_lambda = 0.001  # Regularization parameter
epochs = 100 # Must be equal to k_max/2

# Assuming predefined weights and biases (W, b) and data (X_train, y_train, X_val, y_val)
nns = NeuralNetworkStochastic(weights, biases, learning_rate, l2_lambda)

In [ ]:
# Train the model and visualize metrics
nns.train(X_train_norm, y_train_norm, X_val_norm, y_val_norm, epochs)

In [ ]:
# Access the stored losses and accuracies
train_losses = nn.train_losses  # List of training losses
val_losses = nn.val_losses      # List of validation losses
train_r2_scores = nn.train_r2_scores  # List of training R² scores
val_r2_scores = nn.val_r2_scores      # List of validation R² scores

# Print the final losses and R² scores
print("Final Training Losses:", train_losses[-1])
print("Final Validation Losses:", val_losses[-1])
print("Final Training R² Scores:", train_r2_scores[-1])
print("Final Validation R² Scores:", val_r2_scores[-1])

In [ ]:
# Access the predictions for the training and validation sets
y_train_pred_norm = nns.y_train_pred  # Predicted values for the training set
y_val_pred_norm = nns.y_val_pred      # Predicted values for the validation set

# Denormalize the predictions
y_train_pred = denormalize(y_train_pred_norm, np.min(y_train), np.max(y_train))
y_val_pred = denormalize(y_val_pred_norm, np.min(y_val), np.max(y_val))

# Call the function to plot for training and validation
plot_actual_vs_predicted(y_train, y_train_pred, title="Training: Actual vs Predicted")
plot_actual_vs_predicted(y_val, y_val_pred, title="Validation: Actual vs Predicted")

In [14]:
# Define the number of samples
k_max = 20000
t_test = np.linspace(0, k_max, k_max)  # Time variable

# Generate sinusoidal input for u_test(k)
frequency = 10  # Adjust the frequency as needed
amplitude = 10    # Amplitude of the sine wave
phase = 0         # Phase shift of the sine wave
u_test = amplitude * np.sin(2 * np.pi * frequency * t_test + phase).astype(np.float32)

# Initialize y_test(k) array with dtype float32
y_test = np.zeros(k_max, dtype=np.float32)

# Define y_test(k) calculations with dtype float32
for k in range(1, k_max):
    y_test[k] = np.float32(1 / (1 + (y_test[k-1])**2)) + np.float32(0.25 * u_test[k]) - np.float32(0.3 * u_test[k-1])

# Shift arrays for y_test(k-1), y_test(k-2), u_test(k-1), u_test(k-2)
y_k_1_test = np.zeros(k_max, dtype=np.float32)
y_k_2_test = np.zeros(k_max, dtype=np.float32)
u_k_1_test = np.zeros(k_max, dtype=np.float32)
u_k_2_test = np.zeros(k_max, dtype=np.float32)

# Shifted arrays for testing
y_k_1_test[1:] = y_test[:-1]
y_k_2_test[2:] = y_test[:-2]
u_k_1_test[1:] = u_test[:-1]
u_k_2_test[2:] = u_test[:-2]

In [ ]:
X_test = np.array([y_k_1_test, y_k_2_test, u_test, u_k_1_test, u_k_2_test]).T
y_test = y_test.reshape(-1, 1)

# Normalize each component of X_test
y_k_1_test_norm = normalize(y_k_1_test)
y_k_2_test_norm = normalize(y_k_2_test)
u_test_norm = normalize(u_test)
u_k_1_test_norm = normalize(u_k_1_test)
u_k_2_test_norm = normalize(u_k_2_test)
y_test_norm = normalize(y_test)

# Combine normalized inputs into X_test_norm
X_test_norm = np.array([y_k_1_test_norm, y_k_2_test_norm, u_test_norm, u_k_1_test_norm, u_k_2_test_norm]).T

y_pred_norm = nn.forward(X_test_norm)

y_pred = denormalize(y_pred_norm, np.min(y_test), np.max(y_test))

plot_actual_vs_predicted(y_test, y_pred, title="Test: Actual vs Predicted")